<a href="https://colab.research.google.com/github/hfnvbh/colab-trt-pose/blob/master/trt-pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch torchvision

In [2]:
!apt-get install tensorrt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 tensorrt : Depends: libnvinfer7 (= 7.0.0-1+cuda10.0) but 7.0.0-1+cuda10.2 is to be installed
            Depends: libnvinfer-plugin7 (= 7.0.0-1+cuda10.0) but 7.0.0-1+cuda10.2 is to be installed
            Depends: libnvparsers7 (= 7.0.0-1+cuda10.0) but 7.0.0-1+cuda10.2 is to be installed
            Depends: libnvonnxparsers7 (= 7.0.0-1+cuda10.0) but 7.0.0-1+cuda10.2 is to be installed
            Depends: libnvinfer-bin (= 7.0.0-1+cuda10.0) but it is not going to be installed
            Depends: libnvinfer-dev (= 7.0.0-1+cuda10.0) but 7.0.0-1+cuda10.2 i

In [0]:
!wget -q https://www.dropbox.com/s/p0sfni1dzb359hk/nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb

In [9]:
!dpkg -i nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb

(Reading database ... 144597 files and directories currently installed.)
Preparing to unpack nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb ...
Unpacking nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427 (1-1) over (1-1) ...
Setting up nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427 (1-1) ...


In [12]:
!apt-key add /var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427/7fa2af80.pub

OK


In [13]:
!apt-get update
!apt-get install -y --no-install-recommends libnvinfer5=5.1.2-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-dev=5.1.2-1+cuda10.0
!apt-get install tensorrt
!apt-get install python3-libnvinfer-dev
!apt-get install uff-converter-tf

Get:1 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  InRelease
Ign:1 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  InRelease
Get:2 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  InRelease
Ign:2 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  InRelease
Get:3 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  Release [574 B]
Get:4 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  Release [574 B]
Get:3 file:/var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216  Release [574 B]
Get:4 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  Release [574 B]
Get:5 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  Release.gpg [836 B]
Get:5 file:/var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427  Release.gpg [836 B]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 file:/var/nv-tenso

In [14]:
!dpkg -l | grep TensorRT

ii  graphsurgeon-tf                                              7.0.0-1+cuda10.0                                  amd64        GraphSurgeon for TensorRT package
ii  libnvinfer-dev                                               7.0.0-1+cuda10.2                                  amd64        TensorRT development libraries and headers
ii  libnvinfer-plugin-dev                                        7.0.0-1+cuda10.2                                  amd64        TensorRT plugin libraries
ii  libnvinfer-plugin7                                           7.0.0-1+cuda10.2                                  amd64        TensorRT plugin libraries
ii  libnvinfer5                                                  5.1.2-1+cuda10.0                                  amd64        TensorRT runtime libraries
ii  libnvinfer7                                                  7.0.0-1+cuda10.2                                  amd64        TensorRT runtime libraries
ii  libnvonnxparsers-dev                         

In [9]:
!nvcc --version
!cat /etc/issue
import tensorflow as tf
print(tf.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Ubuntu 18.04.3 LTS \n \l

2.2.0


In [22]:
!pip3 install tqdm cython pycocotools
!apt-get install python3-matplotlib
!git clone https://github.com/NVIDIA-AI-IOT/trt_pose

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-matplotlib is already the newest version (2.1.1-2ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.
fatal: destination path 'trt_pose' already exists and is not an empty directory.


In [24]:
!cd trt_pose && python3 setup.py install

running install
running bdist_egg
running egg_info
creating trt_pose.egg-info
writing trt_pose.egg-info/PKG-INFO
writing dependency_links to trt_pose.egg-info/dependency_links.txt
writing top-level names to trt_pose.egg-info/top_level.txt
writing manifest file 'trt_pose.egg-info/SOURCES.txt'
/usr/local/lib/python3.6/dist-packages/torch/utils/cpp_extension.py:304: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'trt_pose.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/trt_pose
copying trt_pose/parse_objects.py -> build/lib.linux-x86_64-3.6/trt_pose
copying trt_pose/train.py -> build/lib.linux-x86_64-3.6/trt_pose
copying trt_pose/coco.py -> build/lib.linux-x86_64-

In [0]:
!wget -q https://www.dropbox.com/s/r7ex76ofqsnjkjy/resnet18_baseline_att_224x224_A_epoch_249.pth

In [31]:

import json
import trt_pose.trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

ModuleNotFoundError: ignored

In [0]:

import trt_pose.trt_pose.models